In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os
%matplotlib inline


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array

#inception_weights_path ='https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_tf_dim_ordering_tf_kernels.h5'

In [24]:
!git clone https://github.com/ChandraShekhar162001/Gender-Classifier-Using-Deep-Learning.git

Cloning into 'Gender-Classifier-Using-Deep-Learning'...
remote: Enumerating objects: 58650, done.
remote: Total 58650 (delta 0), reused 0 (delta 0), pack-reused 58650
Receiving objects: 100% (58650/58650), 261.43 MiB | 34.15 MiB/s, done.
Checking out files: 100% (58658/58658), done.


In [ ]:
epochs=50 
lr=1e-3
batch_size=128
data=[]
labels=[]

In [ ]:
size=224 #height/width
  

Image Data Generator

In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.4,height_shift_range=0.4,zoom_range=0.3,rotation_range=20,rescale=1/255)

In [ ]:
test_gen= ImageDataGenerator(rescale=1/255) 

In [ ]:
target_size=(size,size)
target_size

(224, 224)

In [27]:
train_generator= train_datagen.flow_from_directory(directory='/content/male-female-face-dataset/Training',target_size=target_size,batch_size=batch_size,class_mode='binary')

Found 47009 images belonging to 2 classes.


In [28]:
validation_generator= test_gen.flow_from_directory(directory='/content/male-female-face-dataset/Validation',target_size=target_size,batch_size=batch_size,class_mode='binary')

Found 11649 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'female': 0, 'male': 1}

In [ ]:
len(train_generator.classes)

47009

In [ ]:
train_generator.class_mode

'binary'

In [ ]:
x,y =  train_generator.next()

In [ ]:
x[0].shape

(224, 224, 3)

In [ ]:
x[0]

array([[[0.38863635, 0.2756336 , 0.21433403],
        [0.36163828, 0.24947925, 0.18986705],
        [0.33464026, 0.22332488, 0.16540006],
        ...,
        [0.3179872 , 0.16368552, 0.10418173],
        [0.31764707, 0.16470589, 0.10588236],
        [0.31764707, 0.16470589, 0.10588236]],

       [[0.41176474, 0.29803923, 0.23529413],
        [0.41176474, 0.29803923, 0.23529413],
        [0.41176474, 0.29803923, 0.23529413],
        ...,
        [0.31764707, 0.16470589, 0.10588236],
        [0.31764707, 0.16470589, 0.10588236],
        [0.31764707, 0.16470589, 0.10588236]],

       [[0.5882353 , 0.4666667 , 0.39607847],
        [0.56629866, 0.44570497, 0.3760917 ],
        [0.5283326 , 0.40942633, 0.34150043],
        ...,
        [0.31764707, 0.16470589, 0.10588236],
        [0.31764707, 0.16470589, 0.10588236],
        [0.31283605, 0.16133817, 0.10299574]],

       ...,

       [[0.9843138 , 0.91372555, 0.86666673],
        [0.9843138 , 0.91372555, 0.86666673],
        [0.9843138 , 0

Build ML Model

---



In [ ]:
model= Sequential()
model.add(InceptionV3(include_top=False,pooling='avg',weights='imagenet'))
model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(2048,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1,activation='sigmoid'))
model.layers[0].trainable=False


87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization_94 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 batch_normalization_95 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 1024)              2

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
len((train_generator.filenames)),batch_size,len((train_generator.filenames))//batch_size

(47009, 128, 367)

In [ ]:
model.fit(train_generator,steps_per_epoch=len(train_generator.filenames)//batch_size,
          epochs=2,validation_data=validation_generator,validation_steps=len(validation_generator.filenames)//batch_size)

Epoch 1/2
367/367 [==============================] - 546s 1s/step - loss: 0.3791 - accuracy: 0.8463 - val_loss: 0.2129 - val_accuracy: 0.9184
Epoch 2/2
212/367 [================>.............] - ETA: 3:34 - loss: 0.3151 - accuracy: 0.8650

Test Model

In [29]:
img_path='/content/male-female-face-dataset/Validation/female/112944.jpg.jpg'

In [ ]:
#from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
img = load_img(img_path,target_size=(size,size,3))
plt.imshow(img)

In [ ]:
img=img_to_array(img)

In [ ]:
img=img/255.0

In [ ]:
img=img.reshape(1,size,size,3)

In [ ]:
img.shape

In [ ]:
model.predict(img)

In [ ]:
train_generator.class_indices

In [ ]:
def get_classes(data):
  prob=model.predict(data)[0][0]


  if prob<0.5:
    return 'female',(1- prob)
  else:
    return 'male',prob

In [ ]:
get_classes(img)

Real-Time Prediction using Web-Cam


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename=take_photo()
  print('Saved to {}'.format(filename))

  #show the image which was just taken.
  display(Image(filename))
except Exception as err:

  print(str(err))

In [ ]:
def get_prediction(img_path):
  img=load_img(img_path,target_size=(size,size,3))
  plt.imshow(img)
  img=img_to_array(img)
  img=img/255.0
  img=img.reshape(1,size,size,3)

  pred,prob = get_classes(img)
  return pred,prob

In [ ]:
get_prediction('/content/male-female-face-dataset/Validation/male/063438.jpg.jpg')